In [1]:
import os
import sqlite3
import pandas as pd

# Run this to go up a level
os.chdir(os.path.dirname(os.path.abspath('')))

In [2]:
os.getcwd()

'/Users/nikkivanhandel/CS6400/mfgdb/CS6400-MfgDB'

In [3]:
conn = sqlite3.connect("mfgdb")

In [4]:
builds_df = pd.read_sql_query("SELECT * FROM Builds", conn)
builds_df = builds_df.drop(0).reset_index(drop=True)
builds_df = builds_df.rename(columns={'Build ID:': 'BuildID'})

print("Builds Data:")
display(builds_df.head())

Builds Data:


,BuildID,Nickname,Operator,DatePrinted,Customer,BuildPlateType,BuildPlateID,FileLocation,ParameterFileName,Successful,...,EstimatedPowderNeeded,PreBuildNotes,PostBuildNotes,RecoaterType,RecyclingState,DosingBoost,GasFlowVoltage,BuildShiftX,BuildShiftY,PowderID
0,1,Rook,Elliott,2020-02-07,Max,Full,1,C:\Users\sa-saldana\Documents\Training\Custome...,Example Link,False,...,,,,,,,,,,1
1,2,ISO Artifact Build,Elliott,2020-02-07,Max,Full,1,C:\Users\sa-saldana\Documents\Training\Custome...,Example Link,,...,,,,,,,,,,1
2,3,Jaime Hexagons,Elliott,2020-02-07,Max,RBV,1,C:\Users\sa-saldana\Documents\Training\Custome...,Example Link,,...,,,,,,,,,,1
3,4,Beam Width Calibration 2,Elliott,2020-02-07,PMRC,Full,2,C:\Users\sa-saldana\Documents\Builds\B005,B005_01-B005_05,,...,,,,,,,,,,1
4,5,"CT Artifact01, Test supports, GT Logo","Elliott,Max",2020-03-13,,Full,3?,C:\Users\sa-saldana\Documents\Builds\B006,EOS_DirectPart,,...,,,,,,,,,,1


In [5]:
builds_df["Successful"].unique()


array(['False', '', 'True'], dtype=object)

In [6]:
# Sensor Data 
sensors_df = pd.read_sql_query("SELECT * FROM Sensors", conn)

print("Sensors:")
display(sensors_df.head())

Sensors:


,Serial,Name,DataKind,Model,Manufacturer,Resolution,Depth
0,1,cam1,PHX-200S-MC,optical image,Lucid Vision,5472x3648,12
1,2,cam2,PHX-032S-MC,optical image,Lucid Vision,2048x1536,12
2,3,laser power,daq-power,voltage,NIST,0.0001,32


In [7]:
# Sensor Imaging database
sensor_data = pd.read_sql_query("SELECT * FROM SensorData", conn)
display(sensor_data.head())

print(sensor_data.dtypes)

,RecordID,BuildID,SensorSerial,Date,Time
0,1,B058,cam1,2023-01-06,15:36:40
1,2,B058,cam1,2023-01-06,15:36:47
2,3,B058,cam1,2023-01-06,15:36:54
3,4,B058,cam1,2023-01-06,15:41:50
4,5,B058,cam1,2023-01-06,15:41:59


RecordID         int64
BuildID         object
SensorSerial    object
Date            object
Time            object
dtype: object


In [8]:
# Analysis Case: Find images which correspond to failed builds
query = """
      with Failed_Builds as (
            Select BuildID
            From Builds 
            Where Successful = 'False'
            )
       Select F.RecordID,
              S.BuildID, 
              F.SensorSerial, 
              F.Date, 
              F.Time
       From Failed_Builds F 
       Join SensorData S
       On F.BuildID = S.BuildID
"""
# Use case: 

failed_builds = pd.read_sql_query(query, conn)
failed_builds.head()

DatabaseError: Execution failed on sql '
      with Failed_Builds as (
            Select BuildID
            From Builds 
            Where Successful = 'False'
            )
       Select F.RecordID,
              S.BuildID, 
              F.SensorSerial, 
              F.Date, 
              F.Time
       From Failed_Builds F 
       Join SensorData S
       On F.BuildID = S.BuildID
': no such column: F.RecordID

In [21]:
# Analysis Case: Find images taken between date - date 
query2 = """
      Select *
      From SensorData
      WHERE DATE(Date) BETWEEN '2023-01-01' AND '2023-01-15'
"""
# Use case: 

Images_bt_dates = pd.read_sql_query(query2, conn)
display(Images_bt_dates)

,RecordID,BuildID,SensorSerial,Date,Time
0,1,B058,cam1,2023-01-06,15:36:40
1,2,B058,cam1,2023-01-06,15:36:47
2,3,B058,cam1,2023-01-06,15:36:54
3,4,B058,cam1,2023-01-06,15:41:50
4,5,B058,cam1,2023-01-06,15:41:59
...,...,...,...,...,...
823,824,B058,cam2,2023-01-06,21:54:36
824,825,B058,cam2,2023-01-06,21:54:48
825,826,B058,cam2,2023-01-06,21:55:37
826,827,B058,cam2,2023-01-06,21:55:48


In [30]:
# Analysis how long a build took
query3 = """
     with Image_Datetime as (
            Select RecordID, 
                   BuildID, 
                   DATETIME(Date ||' '|| Time) as Date_Time
            From SensorData
            )
      Select BuildID,
             (julianday(max(Date_Time)) - julianday(min(Date_Time))) * 24 as Time_Spent_Hours
      From Image_Datetime
      Group By BuildID
      Order by Time_Spent_Hours DESC  
"""
# Use case: 

Build_Time_Spent = pd.read_sql_query(query3, conn)
display(Build_Time_Spent)


,BuildID,Time_Spent_Hours
0,B102,595.728611
1,B063,526.220556
2,B065,476.557500
3,B076,390.815278
4,B118,361.143889
5,B116,184.360278
6,B101,166.626389
7,B099,145.891667
8,B072,144.048611
9,B058,118.529722


In [ ]:
conn.close()